## [Decision Tree](https://en.wikipedia.org/wiki/Decision_tree_learning)

The decision tree is an old, but still relevant nonlinear learning algorithm. The leaves of the tree represent distinct subsets of the training data. The other nodes compare a given attribute to a threshold value (e.g. is the body temperature < 37 °C). The branches starting from the node are associated with the two possible outcomes of the comparison.

<img src="../_img/decision_tree.jpg" width="400" align="left">

In this notebook, we will prepare the simplest version of the decision tree called [decision stump](https://en.wikipedia.org/wiki/Decision_stump), and we will test it on the Boston Housing data set. Moreover, we will explore the capabilities of scikit-learn's decision tree algorithm.

<img src="../_img/decision_stump.png" width="500" align="left">

**Exercise 2**: Implement the training of the decision stump regressor from scratch, and measure the model's root mean squared error (RMSE) on the full Boston Housing data set!

In [1]:
# Loading the data.
import pandas as pd
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
         'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT', 'MEDV']
df = pd.read_csv('../_data/housing_data.txt', delim_whitespace=True, names=names)
X = df[df.columns[:-1]].values # input matrix
y = df['MEDV'].values          # target vector

In [2]:
import numpy as np

n, d = X.shape
sse_min = np.inf
for j in range(d): # iterate over features
    x = X[:, j] # select j-th column

    # sort input and target by target
    idxs = np.argsort(x)
    x_sorted = x[idxs]
    y_sorted = y[idxs]

    midpoints = (x_sorted[1:] + x_sorted[:-1]) / 2
    for i, t in enumerate(midpoints): # iterate over threshold values
        k = (i + 1)
        yhat_L = y_sorted[:k].mean() # average target on the Left side
        yhat_R = y_sorted[k:].mean() # average target on the Right side
        sse_L = ((yhat_L - y_sorted[:k])**2).sum()
        sse_R = ((yhat_R - y_sorted[k:])**2).sum()
        sse = sse_L + sse_R
        
        if sse < sse_min: # found a better decision than previous best
            sse_min = sse
            j_best, t_best = j, t
            yhat_L_best, yhat_R_best = yhat_L, yhat_R
            print('New best found:', 'SSE =', sse,'| Feature index=', j,'| Split threshold=', t)

New best found: SSE = 42714.138495049505 | Feature index= 0 | Split threshold= 0.007690000000000001
New best found: SSE = 42654.06214285715 | Feature index= 0 | Split threshold= 0.01001
New best found: SSE = 42607.60066733068 | Feature index= 0 | Split threshold= 0.01306
New best found: SSE = 42487.76150099801 | Feature index= 0 | Split threshold= 0.013354999999999999
New best found: SSE = 42237.84195247638 | Feature index= 0 | Split threshold= 0.014065
New best found: SSE = 42153.999678714856 | Feature index= 0 | Split threshold= 0.014355
New best found: SSE = 42111.73079365079 | Feature index= 0 | Split threshold= 0.014700000000000001
New best found: SSE = 41739.21143434343 | Feature index= 0 | Split threshold= 0.015195
New best found: SSE = 41403.70269905533 | Feature index= 0 | Split threshold= 0.016235
New best found: SSE = 41349.519812763305 | Feature index= 0 | Split threshold= 0.017435
New best found: SSE = 41236.26263646922 | Feature index= 0 | Split threshold= 0.01824
New bes

In [3]:
j_best, t_best, sse_min

(5, 6.941, 23376.74038861689)

In [4]:
df.columns[j_best] + ' < ' + str(t_best)

'RM < 6.941'

In [5]:
yhat_L_best, yhat_R_best

(19.933720930232557, 37.238157894736844)

**Exercise 3**: Repeat the previous experiment using scikit-learn!

In [6]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=1)
regressor.fit(X, y)

DecisionTreeRegressor(max_depth=1)

In [7]:
# Internal parameters of the trained model (.tree_.{feature, threshold, value})
print(regressor.tree_.feature)
print(regressor.tree_.threshold)
print(regressor.tree_.value)

[ 5 -2 -2]
[ 6.94099998 -2.         -2.        ]
[[[22.53280632]]

 [[19.93372093]]

 [[37.23815789]]]


**Execrice 4**: Apply 3-fold cross-validation!

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def evaluate(regressor, X, y):
    scores = []
    cross_validation = KFold(3, shuffle=True, random_state=42)
    for tr, te in cross_validation.split(X):
        regressor.fit(X[tr], y[tr])
        yhat = regressor.predict(X)
        score = mean_squared_error(y[te], yhat[te])**0.5
        scores.append(score)
    return np.mean(scores)

**Exercise 5:** Determine what maximal depth gives the lowest RMSE!

In [54]:
# also create a plot

In [14]:
rmses = []
for i in range(1, 21):
    rmse = evaluate(DecisionTreeRegressor(max_depth=i, random_state=42), X, y)
    rmses.append(rmse)
    
rmses

[7.435013307929704,
 5.6748753750814815,
 4.996460734552788,
 4.597959972515105,
 4.47509607287735,
 4.500246228889928,
 4.542081680983131,
 4.6814266725402245,
 4.717507870797422,
 4.563786081100744,
 4.912020155993026,
 4.681568954338988,
 4.788222129444225,
 4.781339830545176,
 4.647130972747677,
 4.948232662874149,
 4.891792341067529,
 4.980090429380394,
 4.877156332064531,
 4.878312499423474]

**Exercise 6**: Train a decision tree of depth 3 and visualize the trained model!

In [15]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

re = DecisionTreeRegressor()

In [ ]:
# Number of parameters in the tree.


In [ ]:
# Total size of the data set.


## Decision Trees for Classification

- Decision trees can also be applied to classification problems.


- The necessary modification is that instead of sum of squared error, a different split criterion should be applied (e.g. misclassification count, [Gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity), [information gain](https://en.wikipedia.org/wiki/Decision_tree_learning#Information_gain)), and the leaf predictions should be changed to class probabilities.


- Decision trees can handle multiclass problems too.

**Exercise 7**: Apply a decision tree classifier for the Wisconsin Breast Cancer data set! Use 5-fold cross-validation! The evaluation metric should be the ratio of correct classifications. Determine the maximal depth that gives the highest accuracy! Compare the best decision tree against logistic regression!

In [ ]:
# Load the data to DataFrame.
import pandas as pd
names = [
    'Sample_code_number', 'Clump_Thickness', 'Uniformity_of_Cell_Size',
    'Uniformity_of_Cell_Shape', 'Marginal_Adhesion', 'Single_Epithelial_Cell_Size',
    'Bare_Nuclei', 'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'
]
df = pd.read_csv('../_data/wisconsin_data.txt', sep=',', names=names, na_values='?')
df['Bare_Nuclei'].fillna(0, inplace=True)
df.head()

## Decision Trees vs. Linear Models

Decision trees
- ...are insensitive to the scale of the input features 😀
- ...are easier to explain 😀
- ...can learn complex patterns 😀
- ...don not handle sparse data efficiently ☹️
- ...tend to overfit more ☹️